In [1]:
# Carregando as bibliotecas

import pandas as pd
import numpy as np
import yfinance as yf
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split 

get_ipython().run_line_magic("matplotlib", "inline")
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Carregando a base de dados e fazendo os checks iniciais

# Carrega as bases
ticker1 = "^BVSP" 
df1 = yf.download(ticker1, "2015-01-01", "2022-12-31")
 
ticker2 = "BRL=X" 
df2 = yf.download(ticker2, "2015-01-01", "2022-12-31")

# Parâmetros do algoritmo
periodos = 1

# df1
df1["Retorno"] = df1["Adj Close"].pct_change(periodos)
df1["Fech_Ant"] = df1["Close"].shift(1)
df1["Alvo"] = df1["Retorno"].shift(-1)
df1["Candle_Atual"] = np.where(df1["Retorno"] > 0, 1, 0)
df1["Candle_A1"] = df1["Candle_Atual"].shift(1)
df1["Candle_A2"] = df1["Candle_A1"].shift(1)
df1["Candle_A3"] = df1["Candle_A2"].shift(1)
df1 = df1.dropna(axis = 0) 

# df2
df2["Retorno"] = df2["Adj Close"].pct_change(periodos)
df2["Fech_Ant"] = df2["Close"].shift(1)
df2["Alvo"] = df2["Retorno"].shift(-1)
df2["Candle_Atual"] = np.where(df2["Retorno"] > 0, 1, 0)
df2["Candle_A1"] = df2["Candle_Atual"].shift(1)
df2["Candle_A2"] = df2["Candle_A1"].shift(1)
df2["Candle_A3"] = df2["Candle_A2"].shift(1)
df2 = df2.dropna(axis = 0) 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
print(df1["Candle_Atual"].describe()*100)
print()
print(df2["Candle_Atual"].describe()*100)

count    183700.000000
mean         52.367991
std          49.957494
min           0.000000
25%           0.000000
50%         100.000000
75%         100.000000
max         100.000000
Name: Candle_Atual, dtype: float64

count    193700.000000
mean         51.006711
std          50.002773
min           0.000000
25%           0.000000
50%         100.000000
75%         100.000000
max         100.000000
Name: Candle_Atual, dtype: float64


In [5]:
# Agora fazemos a análise se os dois últimos candles são de alta ou de baixa

df1["Dois_Ult_Alta"] = np.where(((df1["Candle_Atual"] == 1) & (df1["Candle_A1"] == 1)), 1, 0)
df2["Dois_Ult_Alta"] = np.where(((df2["Candle_Atual"] == 1) & (df2["Candle_A1"] == 1)), 1, 0)

df1["Dois_Ult_Baixa"] = np.where(((df1["Candle_Atual"] == 0) & (df1["Candle_A1"] == 0)), 1, 0)
df2["Dois_Ult_Baixa"] = np.where(((df2["Candle_Atual"] == 0) & (df2["Candle_A1"] == 0)), 1, 0)

In [6]:
print(df1["Dois_Ult_Alta"].describe()*100)
print()
print(df2["Dois_Ult_Alta"].describe()*100)

count    183700.000000
mean         26.510615
std          44.150990
min           0.000000
25%           0.000000
50%           0.000000
75%         100.000000
max         100.000000
Name: Dois_Ult_Alta, dtype: float64

count    193700.000000
mean         23.851316
std          42.628447
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max         100.000000
Name: Dois_Ult_Alta, dtype: float64


In [7]:
print(df1["Dois_Ult_Baixa"].describe())
print()
print(df2["Dois_Ult_Baixa"].describe())

count    1837.000000
mean        0.217202
std         0.412453
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: Dois_Ult_Baixa, dtype: float64

count    1937.000000
mean        0.218895
std         0.413604
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: Dois_Ult_Baixa, dtype: float64


In [8]:
# Cria uma média móvel da direção dos candles
p = 10

#### df1
df1["mm_atual"] = df1["Candle_Atual"].rolling(p).mean()
df1 = df1.dropna(axis = 0) 

#### df2
df2["mm_atual"] = df2["Candle_Atual"].rolling(p).mean()
df2 = df2.dropna(axis = 0) 


#### df1
df1["mm_dois_ult_alta"] = df1["Dois_Ult_Alta"].rolling(p).mean()
df1 = df1.dropna(axis = 0) 

#### df2
df2["mm_dois_ult_alta"] = df2["Dois_Ult_Alta"].rolling(p).mean()
df2 = df2.dropna(axis = 0) 

#### df1
df1["mm_dois_ult_baixa"] = df1["Dois_Ult_Baixa"].rolling(p).mean()
df1 = df1.dropna(axis = 0) 

#### df2
df2["mm_dois_ult_baixa"] = df2["Dois_Ult_Baixa"].rolling(p).mean()
df2 = df2.dropna(axis = 0) 


In [11]:
fig = make_subplots(rows = 3, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Scatter(x = df1.index, y = df1["Adj Close"]
                         , name = ticker1, line = dict(color = "blue"))
              , row = 1, col = 1)

fig.add_trace(go.Scatter(x = df1.index, y = df1["mm_atual"]*100
                         , name = "MM Dir " + ticker1, line = dict(color = "blue"))
              , row = 2, col = 1)

fig.add_trace(go.Scatter(x = df2.index, y = df2["mm_atual"]*100
                         ,  name = "MM Dir " + ticker2, line = dict(color = "black"))
              , row = 2, col = 1)

fig.add_trace(go.Scatter(x = df1.index, y = df1["mm_dois_ult_alta"]*100
                         , name = "MM 2 Ult Alta " + ticker1, line = dict(color = "blue"))
              , row = 3, col = 1)

fig.add_trace(go.Scatter(x = df2.index, y = df2["mm_dois_ult_alta"]*100
                         ,  name = "MM 2 Ult Alta " + ticker2, line = dict(color = "black"))
              , row = 3, col = 1)


fig.update_layout(height = 1200, width = 900
                  , title_text = "Direção dos candles - Outspoken Market"
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis3_title = "Year"
                  , yaxis_title = "Ibovespa"
                  , yaxis2_title = "MM da Direção do Candle"
                  , yaxis3_title = "MM dos 2 ult. dias"
                  , legend_title = "Study objects"
                  , font = dict(size = 15, color = "Black")
                 )

fig.update_layout(
    xaxis = dict(
        rangeselector = dict(
            buttons = [
                dict(count = 4,
                     label = "4m",
                     step = "month",
                     stepmode = "backward"),
                dict(count = 6,
                     label = "6m",
                     step = "month",
                     stepmode = "backward"),
                dict(count = 1,
                     label = "1y",
                     step = "year",
                     stepmode = "backward"),
                dict(step = "all")
            ]),
        type = "date")
    , xaxis3_rangeslider_visible = True
    , xaxis3_type = "date"
    , yaxis = dict(autorange = True, fixedrange = False)
    , yaxis2= dict(autorange = True, fixedrange = False)
    , yaxis3= dict(autorange = True, fixedrange = False)
    )
fig.update_layout(hovermode = "x")

fig.show()

In [ ]:
df1.columns

In [12]:
# Separando os dados entre treinamento e teste

# Vamos treinar o modelo de 2015 a 2018
start_train = "2015-01-01"
end_train = "2018-12-31"

# Vamos testar o modelo de 2018 a 2022
start_test = "2019-01-01"
end_test = "2022-12-31"

df1_train = df1.loc[start_train : end_train]

df1_test = df1.loc[start_test : end_test]

# Separando os dados com as variaveis em x e o alvo em y
 
x_train = df1_train.iloc[:, 13:18]
y_train = df1_train["Alvo"]

x_test = df1_test.iloc[:, 13:18]
y_test = df1_test["Alvo"]

In [13]:
# Treinando o modelo com statsmodels

# Nota importante: por padrão o statsmodels não acrescenta o intercepto
# você deve acrescentá-lo manualmente

x_train_ = sm.add_constant(x_train)
x_test_ = sm.add_constant(x_test)

lr_sm = sm.OLS(y_train, x_train_).fit() # Linha que treina o modelo

y_pred_train = lr_sm.predict(x_train_)

y_pred_test = lr_sm.predict(x_test_)

# Mostra as estatísticas 
lr_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Alvo   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.1782
Date:                Sun, 12 Jun 2022   Prob (F-statistic):              0.971
Time:                        16:44:47   Log-Likelihood:                 2700.2
No. Observations:                 957   AIC:                            -5388.
Df Residuals:                     951   BIC:                            -5359.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.0033      0.008     -0.405      0.686      -0.019       0.013
Dois_Ult_Alta        -0.0008      0.001     -0.661      0.509      -0.003       0.002
Dois_Ult_Baixa        0.0002      0.001      0.152      0.879      -0.002       0.003
mm_atual              0.0062      0.016      0.393      0.695      -0.025       0.037
mm_dois_ult_alta     -0.0004      0.008     -0.045      0.964      -0.017       0.016
mm_dois_ult_baixa     0.0045      0.008      0.540      0.589      -0.012       0.021
==============================================================================
Omnibus:                       54.639   Durbin-Watson:                   1.975
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              201.447
Skew:                          -0.056   Prob(JB):                     1.80e-44
Kurtosis:                       5.245   Cond. No.                         54.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""